In [ ]:
!pip install opendatasets

In [ ]:
from __future__ import print_function
import argparse
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from sklearn.model_selection import train_test_split
import opendatasets as od
from sklearn.impute import SimpleImputer
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df_scaled=data_preprocess()
X_train, X_test = train_test_split(df_scaled, test_size=0.3)

In [ ]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(4, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 4)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        if self.training:
            std = torch.exp(0.5*logvar)
            eps = torch.randn_like(std)
            return eps.mul(std).add_(mu)
        else:
            return mu

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return F.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 4))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
def loss_function(recon_x, x, mu, logvar):
    MSE = F.mse_loss(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return MSE + KLD

In [ ]:
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32))
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

def train(epoch,model=model, train_loader=train_loader, optimizer=optimizer):
    model.train()
    for epoch in range(epoch):
        train_loss = 0
        for batch_idx, data in enumerate(train_loader):
            x = data[0]
            optimizer.zero_grad()
            recon_batch, mu, logvar = model(x)
            loss = loss_function(recon_batch, x, mu, logvar)
            loss.backward()
            train_loss += loss.item()
            optimizer.step()
    if epoch%100==0:
      print('Epoch {}: Average Train Loss: {:.4f}'.format(epoch+1, train_loss / len(train_loader.dataset)))

In [ ]:
for epoch in range(1, 1000):
  train(epoch)

Epoch 1: Average Train Loss: 0.3565
Epoch 101: Average Train Loss: 0.3515
Epoch 201: Average Train Loss: 0.3519
